# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
print(y[:5])



[0 0 0 1 0]


In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1).values

# Review the features DataFrame
print(X[:5, :])



[[7.400e+00 7.000e-01 0.000e+00 1.900e+00 7.600e-02 1.100e+01 3.400e+01
  9.978e-01 3.510e+00 5.600e-01 9.400e+00]
 [7.800e+00 8.800e-01 0.000e+00 2.600e+00 9.800e-02 2.500e+01 6.700e+01
  9.968e-01 3.200e+00 6.800e-01 9.800e+00]
 [7.800e+00 7.600e-01 4.000e-02 2.300e+00 9.200e-02 1.500e+01 5.400e+01
  9.970e-01 3.260e+00 6.500e-01 9.800e+00]
 [1.120e+01 2.800e-01 5.600e-01 1.900e+00 7.500e-02 1.700e+01 6.000e+01
  9.980e-01 3.160e+00 5.800e-01 9.800e+00]
 [7.400e+00 7.000e-01 0.000e+00 1.900e+00 7.600e-02 1.100e+01 3.400e+01
  9.978e-01 3.510e+00 5.600e-01 9.400e+00]]


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
from sklearn.model_selection import train_test_split

# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)




### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Scale the features testing dataset
X_test_scaled = scaler.transform(X_test)



---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the number of features
number_of_features = X_train_scaled.shape[1]


# Review the number of features
number_of_features


11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_neurons = 1


In [11]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_of_features))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))



In [12]:
# Display the Sequential model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, verbose=1)


Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 0.6735 - accuracy: 0.5571
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6482 - accuracy: 0.6113
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6257 - accuracy: 0.6806
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6058 - accuracy: 0.7048
Epoch 5/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5875 - accuracy: 0.7214
Epoch 6/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5719 - accuracy: 0.7206
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5588 - accuracy: 0.7314
Epoch 8/50
38/38 [==============================] - 0s 5ms/step - loss: 0.5475 - accuracy: 0.7356
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5373 - accuracy: 0.7389
Epoch 10/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5293 - accuracy: 0.7465
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")

13/13 - 0s - loss: 0.5178 - accuracy: 0.7550 - 448ms/epoch - 34ms/step
Loss: 0.51775723695755, Accuracy: 0.7549999952316284


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
file_path = "student_loans.keras"

# Export your model to a keras file
model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
from tensorflow.keras.models import load_model

# Set the model's file path
file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
import numpy as np
import pandas as pd

# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[:5])


13/13 [==============================] - 0s 2ms/step
[[0.39401084]
 [0.30171463]
 [0.894347  ]
 [0.67804056]
 [0.9869727 ]]


In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=['Prediction'])
predictions_df['Prediction'] = predictions_df['Prediction'].apply(lambda x: 1 if x >= 0.5 else 0)

# Display the first few rows of the DataFrame
print(predictions_df.head())


   Prediction
0           0
1           0
2           1
3           1
4           1


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['Prediction']))



              precision    recall  f1-score   support

           0       0.72      0.79      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.75       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.



1. **Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students.**

To build an effective recommendation system for student loan options, you would need to collect a variety of data including:

- **Student Information:** Academic performance, major/field of study, demographic data, financial need, part-time or full-time enrollment status.
- **Loan Information:** Interest rates, repayment terms, lender reputation, federal vs. private loans, deferment options.
- **Outcome Data:** Historical data on loan repayment success rates among students with similar profiles, including default rates.

This data is relevant because it can help match students with loan options that best fit their financial needs and academic circumstances, potentially improving repayment success rates.

2. **Based on the chosen data, which filtering method would be most suitable?**

Given the diversity and specificity of the data involved, a **content-based filtering** method would be appropriate. This method can utilize detailed information about both students and loan options to make personalized recommendations. For instance, students pursuing degrees with higher expected earnings (like STEM fields) might be matched with different loan options than those in fields with lower starting salaries.

3. **Real-world challenges in building a recommendation system for student loans:**

- **Privacy and Data Security:** Handling sensitive personal and financial information requires robust security measures to protect against data breaches.
- **Data Accuracy and Bias:** Ensuring the data used for recommendations is accurate and unbiased is crucial. Biased data could lead to unfair loan recommendations, disproportionately affecting certain student groups.

These challenges are significant because they directly impact the fairness, effectiveness, and trustworthiness of the recommendation system, potentially influencing students' financial futures.
